In [ ]:
import os

os.chdir("/home/dahala/Transformer-Gorkha/")
import tensorflow as tf
import json
import numpy as np
import seaborn as sns
from src import preparedata
from src import transformermodel
from src import traintransformer
import matplotlib.pyplot as plt
from tensorflow.keras import layers, optimizers, losses, metrics, Model
from tqdm.notebook import tqdm

print("json imported")

In [ ]:
params = json.load(open("params/paramsANN.json", "r"))
dataset = preparedata.readTransformerData(params["dataprepinargs"])
dataset.preparedata()

In [ ]:
# predict with timeseriesandconstant
model = tf.keras.models.load_model("checkpoints/timeseriesandconstant")

In [ ]:
model.summary()

# Estimate effects using gradients to obtain the SHAP equivalent

In [ ]:
# trail for Xtest
num_features = dataset.Xt.shape[2]
num_instances = dataset.Xt.shape[0]
all_scores = []
# calculate baseline gradients
batch_size = 10
batches = int(num_instances / batch_size)

for i in tqdm(range(batches)):
    start = batch_size * i
    if i != (batches):
        end = batch_size * (i + 1)
    else:
        end = num_instances
    dyn = dataset.Xt[start:end, :, :]
    stt = np.nan_to_num(dataset.Xc[start:end, :-1], 0)
    with tf.GradientTape() as tape_baseline:
        bb = tf.convert_to_tensor(np.zeros(dyn.shape))
        tape_baseline.watch(bb)
        output_baseline = model([bb, stt])  # Assuming a binary classification task
        gradients_baseline = tape_baseline.gradient(
            output_baseline, bb
        )  # Assuming a binary classification task

    with tf.GradientTape() as tape_instance:
        xx = tf.convert_to_tensor(dyn)
        tape_instance.watch(xx)
        output = model([xx, stt])  # Assuming a binary classification task
        gradients_instances = tape_instance.gradient(
            output, xx
        )  # Assuming a binary classification task
    scoresv2 = (
        (xx - bb)
        * gradients_instances
        / (gradients_instances - gradients_baseline + 1e-10)
    )
    del gradients_instances, gradients_baseline
    all_scores.append(scoresv2.numpy())
    del scoresv2

In [ ]:
all_scores_3d = np.array(all_scores).reshape(dataset.Xt.shape)

In [ ]:
np.save(f"Data/ExplainedGradients/all_scores_3d.npy", all_scores_3d)

In [ ]:
def savescoreaxis(axis=0):
    if axis == "all":
        all_scores = np.nanmean(all_scores_3d, axis=2)
        all_gm = np.nanmax(dataset.Xt, axis=2)
        max_locs = np.argmax(all_gm, axis=1)
        print(all_scores.shape, all_gm.shape, max_locs.shape)
    else:
        all_scores = all_scores_3d[:, :, axis]
        all_gm = dataset.Xt[:, :, axis]
        max_locs = np.argmax(all_gm, axis=1)

    # for each gm waveform, calculate the effect in each direction
    segpeak = 30
    pre = []
    post = []
    pregm = []
    postgm = []
    for i in tqdm(range(all_scores.shape[0])):
        maxloc = max_locs[i]
        gm = all_gm[i]
        score = all_scores[i, :]
        tmpprescore = [score[:maxloc][: -int(maxloc / segpeak) * segpeak].mean()]
        tmpprescore.extend(
            score[:maxloc][-int(maxloc / segpeak) * segpeak :]
            .reshape(-1, segpeak)
            .mean(axis=1)
        )

        tmppostscore = [
            score[maxloc:][int((score.shape[0] - maxloc) / segpeak) * segpeak :].mean()
        ]
        tmppostscore = (
            score[maxloc:][-int((score.shape[0] - maxloc) / segpeak) * segpeak :]
            .reshape(-1, segpeak)
            .mean(axis=1)
            .tolist()
            + tmppostscore
        )

        pre.append(tmpprescore)
        post.append(tmppostscore)
        if (np.isnan(tmpprescore).sum() != 0) and (np.isnan(tmppostscore).mean() != 0):
            print(i)

        temmppregm = [gm[:maxloc][: -int(maxloc / segpeak) * segpeak].mean()]
        temmppregm.extend(
            gm[:maxloc][-int(maxloc / segpeak) * segpeak :]
            .reshape(-1, segpeak)
            .mean(axis=1)
        )

        temppostgm = [
            gm[maxloc:][int((score.shape[0] - maxloc) / segpeak) * segpeak :].mean()
        ]
        temppostgm = (
            temppostgm
            + gm[maxloc:][-int((score.shape[0] - maxloc) / segpeak) * segpeak :]
            .reshape(-1, segpeak)
            .mean(axis=1)
            .tolist()
        )
        pregm.append(temmppregm)
        postgm.append(temppostgm)
        # break

    list_len = [len(i) for i in pre]
    maxlen = max(list_len)
    print(maxlen)
    pre2 = [np.zeros(maxlen - len(i)).tolist() + i for i in pre]
    pre2 = np.array(pre2)

    list_len = [len(i) for i in post]
    maxlen = max(list_len)
    print(maxlen)
    post2 = [i + np.zeros(maxlen - len(i)).tolist() for i in post]
    post2 = np.array(post2)

    list_len = [len(i) for i in pregm]
    maxlen = max(list_len)
    print(maxlen)
    pregm2 = [np.zeros(maxlen - len(i)).tolist() + i for i in pregm]
    pregm2 = np.array(pregm2)

    list_len = [len(i) for i in postgm]
    maxlen = max(list_len)
    print(maxlen)
    postgm2 = [i + np.zeros(maxlen - len(i)).tolist() for i in postgm]
    postgm2 = np.array(postgm2)

    # save all data
    np.save(f"Data/ExplainedGradients/prepeakgm_{axis}.npy", pregm2)
    np.save(f"Data/ExplainedGradients/postpeakgm_{axis}.npy", postgm2)
    np.save(f"Data/ExplainedGradients/prepeakshap_{axis}.npy", pre2)
    np.save(f"Data/ExplainedGradients/postpeakshap_{axis}.npy", post2)


# savescoreaxis(0)
# savescoreaxis(1)
# savescoreaxis(2)
savescoreaxis("all")